# Loop 6 Analysis: Understanding the Gap to Target

## Current Situation
- Best CV score: 70.681004 (exp_005)
- Best LB score: 70.682741 (exp_002)
- Target: 68.922808
- Gap: 1.758 points (2.5%)

## Key Findings from Previous Experiments
1. Pre-optimized baseline (70.68) is at a very strong local optimum
2. All local search methods (SA, swaps, squeeze, backward propagation) found minimal improvements
3. Only backward propagation found 0.001737 points improvement
4. The baseline cannot be improved with local search methods

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import os

getcontext().prec = 30
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
# Check what pre-optimized CSVs we have access to
import glob

csv_files = glob.glob('/home/code/**/*.csv', recursive=True)
print(f"Found {len(csv_files)} CSV files")

# Group by directory
from collections import defaultdict
by_dir = defaultdict(list)
for f in csv_files:
    d = os.path.dirname(f)
    by_dir[d].append(os.path.basename(f))

for d, files in sorted(by_dir.items()):
    print(f"\n{d}:")
    for f in sorted(files)[:5]:
        print(f"  {f}")
    if len(files) > 5:
        print(f"  ... and {len(files)-5} more")

Found 29 CSV files

/home/code/experiments/001_baseline:
  submission.csv
  submission_optimized.csv

/home/code/experiments/002_grid_baseline:
  submission.csv
  submission_final.csv
  submission_optimized.csv
  submission_optimized2.csv
  submission_optimized3.csv
  ... and 1 more

/home/code/experiments/003_preoptimized:
  baseline.csv
  ensemble_submission.csv
  repaired_baseline.csv
  submission.csv
  submission_backprop.csv
  ... and 3 more

/home/code/experiments/004_jonathanchan_optimizer:
  submission.csv
  submission_final.csv
  submission_opt.csv
  submission_repaired.csv

/home/code/experiments/005_python_sa_optimizer:
  submission.csv

/home/code/experiments/006_grid_translation:
  submission.csv

/home/code/experiments/006_grid_translation_results:
  submission.csv

/home/code/submission_candidates:
  candidate_000.csv
  candidate_001.csv
  candidate_002.csv
  candidate_003.csv
  candidate_004.csv
  ... and 1 more


In [3]:
# Check the snapshot directory for pre-optimized baselines
import os

snapshot_dir = '/home/nonroot/snapshots/santa-2025'
if os.path.exists(snapshot_dir):
    print(f"Snapshot directory exists: {snapshot_dir}")
    for root, dirs, files in os.walk(snapshot_dir):
        csv_files = [f for f in files if f.endswith('.csv')]
        if csv_files:
            print(f"\n{root}:")
            for f in csv_files[:10]:
                print(f"  {f}")
else:
    print(f"Snapshot directory does not exist: {snapshot_dir}")

Snapshot directory exists: /home/nonroot/snapshots/santa-2025

/home/nonroot/snapshots/santa-2025/21116303805/code:
  submission.csv

/home/nonroot/snapshots/santa-2025/21116303805/code/experiments/004_sa_v1_parallel:
  submission_best.csv
  submission_v18.csv

/home/nonroot/snapshots/santa-2025/21116303805/code/experiments/005_backward_propagation:
  optimized.csv

/home/nonroot/snapshots/santa-2025/21116303805/code/experiments/002_preoptimized:
  submission.csv

/home/nonroot/snapshots/santa-2025/21116303805/code/submission_candidates:
  candidate_000.csv
  candidate_002.csv
  candidate_003.csv
  candidate_001.csv

/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized:
  ensemble.csv
  submission.csv
  santa-2025.csv
  best_ensemble.csv

/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram:
  72.49.csv
  71.97.csv

/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/telegram_extracted:
  72.49.csv
  71.97.csv

/home/nonroot/snapsh

In [12]:
# Analyze the current best submission
df = pd.read_csv('/home/submission/submission.csv')
print(f"Submission shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

# Count configurations per N
n_counts = df['id'].apply(lambda x: int(x.split('_')[0])).value_counts().sort_index()
print(f"\nN values: {n_counts.index.min()} to {n_counts.index.max()}")
print(f"Total configurations: {len(n_counts)}")

Submission shape: (20100, 4)

First few rows:
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  

N values: 1 to 200
Total configurations: 200


In [9]:
# Check scores of all pre-optimized baselines in the snapshot directory
import os

snapshot_base = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized'

# List all CSV files
preoptimized_csvs = []
for root, dirs, files in os.walk(snapshot_base):
    for f in files:
        if f.endswith('.csv'):
            preoptimized_csvs.append(os.path.join(root, f))

print(f"Found {len(preoptimized_csvs)} pre-optimized CSVs:")
for csv_path in sorted(preoptimized_csvs):
    print(f"  {csv_path}")

Found 30 pre-optimized CSVs:
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/blended/submission (77).csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/bucket-of-chump/submission.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/chistyakov/submission_best.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-try3/submission.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-try3/submission_sa.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/New_Tree_144_196.csv
  /home/nonroot/snapshots/santa-20

In [10]:
# Calculate scores for all pre-optimized CSVs (quick check without overlap validation)
def quick_score(csv_path):
    """Calculate score without overlap validation (for quick comparison)."""
    try:
        df = pd.read_csv(csv_path)
        total_score = 0
        for n in range(1, 201):
            group_data = df[df["id"].str.startswith(f"{n:03d}_")]
            if len(group_data) == 0:
                continue
            
            # Parse coordinates
            xs = []
            ys = []
            for _, row in group_data.iterrows():
                x_str = str(row["x"])
                y_str = str(row["y"])
                if x_str.startswith('s'):
                    x_str = x_str[1:]
                if y_str.startswith('s'):
                    y_str = y_str[1:]
                try:
                    xs.append(float(x_str))
                    ys.append(float(y_str))
                except:
                    pass
            
            if not xs:
                continue
            
            # Approximate bounding box (ignoring tree shape for speed)
            # This is a rough estimate - actual score will be slightly different
            min_x, max_x = min(xs), max(xs)
            min_y, max_y = min(ys), max(ys)
            side = max(max_x - min_x + 0.7, max_y - min_y + 1.0)  # Add tree dimensions
            score = side ** 2 / n
            total_score += score
        
        return total_score
    except Exception as e:
        return None

print("Calculating quick scores for all pre-optimized CSVs...")
scores = {}
for csv_path in sorted(preoptimized_csvs):
    score = quick_score(csv_path)
    if score:
        scores[csv_path] = score
        name = os.path.basename(csv_path)
        print(f"  {name}: ~{score:.2f}")

print(f"\nBest quick scores:")
for path, score in sorted(scores.items(), key=lambda x: x[1])[:10]:
    print(f"  {os.path.basename(path)}: ~{score:.2f}")

Calculating quick scores for all pre-optimized CSVs...


  best_ensemble.csv: ~86.92


  submission (77).csv: ~87.64


  submission.csv: ~86.89


  submission_best.csv: ~87.26


...

  submission.csv: ~86.89


  71.97.csv: ~87.40


  72.49.csv: ~88.51


  71.97.csv: ~87.40


  72.49.csv: ~88.51

Best quick scores:
  submission_opt1.csv: ~86.19
  submission.csv: ~86.89
  submission.csv: ~86.89
  best_ensemble.csv: ~86.92
  ensemble.csv: ~86.92
  santa-2025.csv: ~86.92
  santa-2025.csv: ~86.92
  submission_best.csv: ~87.26
  submission_70_926149550346.csv: ~87.26
  submission_70_936673758122.csv: ~87.36


In [11]:
# Calculate ACTUAL scores for the most promising CSVs using proper tree geometry
# Focus on the ones with best quick scores

promising_csvs = [
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/bucket-of-chump/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/submission_opt1.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/71.97.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/submission_70_926149550346.csv',
]

print("Calculating ACTUAL scores for promising CSVs...")
actual_scores = {}

for csv_path in promising_csvs:
    try:
        df = pd.read_csv(csv_path)
        total_score = 0
        overlap_count = 0
        
        for n in range(1, 201):
            trees = load_configuration_from_df(n, df)
            if trees:
                score = get_score(trees, n)
                total_score += score
        
        actual_scores[csv_path] = total_score
        name = os.path.basename(csv_path)
        print(f"  {name}: {total_score:.6f}")
    except Exception as e:
        print(f"  {os.path.basename(csv_path)}: ERROR - {e}")

print(f"\nBest actual scores:")
for path, score in sorted(actual_scores.items(), key=lambda x: x[1])[:5]:
    print(f"  {os.path.basename(path)}: {score:.6f}")

Calculating ACTUAL scores for promising CSVs...
  submission.csv: ERROR - name 'load_configuration_from_df' is not defined
  santa-2025.csv: ERROR - name 'load_configuration_from_df' is not defined
  submission_opt1.csv: ERROR - name 'load_configuration_from_df' is not defined
  best_ensemble.csv: ERROR - name 'load_configuration_from_df' is not defined
  71.97.csv: ERROR - name 'load_configuration_from_df' is not defined
  submission_70_926149550346.csv: ERROR - name 'load_configuration_from_df' is not defined

Best actual scores:


In [13]:
# Calculate score breakdown by N
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

def get_side_length(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1]) / float(scale_factor)

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_side_length(trees)
    return side ** 2 / n

def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

print("Helper functions defined")

Helper functions defined


In [14]:
# Calculate score breakdown
scores = {}
for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    if trees:
        scores[n] = get_score(trees, n)

total_score = sum(scores.values())
print(f"Total score: {total_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap: {total_score - 68.922808:.6f} points")

# Find N values with highest scores (most room for improvement)
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
print(f"\nTop 10 N values with highest scores:")
for n, score in sorted_scores[:10]:
    print(f"  N={n}: score={score:.6f}")

Total score: 70.681004
Target: 68.922808
Gap: 1.758196 points

Top 10 N values with highest scores:
  N=1: score=0.661250
  N=2: score=0.450779
  N=3: score=0.434745
  N=5: score=0.416850
  N=4: score=0.416545
  N=7: score=0.399897
  N=6: score=0.399610
  N=9: score=0.387415
  N=8: score=0.385407
  N=15: score=0.379203


In [ ]:
# Calculate what improvement is needed per N to reach target
target = 68.922808
current = total_score
gap = current - target

print(f"Gap to close: {gap:.6f} points")
print(f"Average improvement needed per N: {gap/200:.6f} points")

# If we could improve the top 20 N values by 10%, how much would we gain?
top_20_score = sum([s for n, s in sorted_scores[:20]])
print(f"\nTop 20 N values contribute: {top_20_score:.6f} points ({100*top_20_score/current:.1f}%)")
print(f"If improved by 10%: would save {top_20_score * 0.1:.6f} points")

# What if we could improve ALL N values by a uniform percentage?
required_improvement = gap / current
print(f"\nRequired uniform improvement: {100*required_improvement:.2f}%")

In [ ]:
# Analyze the theoretical lower bound
# For a single tree, the minimum bounding box is the tree's own bounding box
# For N trees, the theoretical minimum is when trees are perfectly packed

# Tree dimensions (approximate)
tree_width = 0.7  # base width
tree_height = 1.0  # from trunk bottom to tip

# For N=1, the minimum score is tree_height^2 / 1 = 1.0
# But the actual score is 0.661, which means the tree is rotated to minimize bbox

print("Theoretical analysis:")
print(f"Tree dimensions: {tree_width} x {tree_height}")
print(f"Tree area: {tree_width * tree_height:.4f}")

# For perfect packing, the minimum bounding box area would be N * tree_area
# But trees are irregular, so there's always some wasted space

# Calculate packing efficiency for each N
print(f"\nPacking efficiency by N:")
for n in [1, 10, 50, 100, 200]:
    if n in scores:
        side = np.sqrt(scores[n] * n)  # side^2/n = score, so side = sqrt(score*n)
        box_area = side ** 2
        tree_total_area = n * tree_width * tree_height
        efficiency = tree_total_area / box_area
        print(f"  N={n}: side={side:.4f}, efficiency={100*efficiency:.1f}%")

In [ ]:
# Key insight: The gap of 1.76 points represents ~2.5% improvement needed
# This is a significant gap that cannot be closed with local search

# What approaches haven't been tried?
print("Approaches tried:")
print("1. Greedy placement - baseline 164.09")
print("2. Grid placement (zaburo) - improved to 88.14")
print("3. Pre-optimized baseline - improved to 70.68")
print("4. bbox3 C++ optimizer - no improvement")
print("5. fix_direction rotation - no improvement")
print("6. jonathanchan SA optimizer - introduced overlaps")
print("7. Pure Python SA with Shapely - no improvement")
print("8. Grid translation (egortrushin) - worse than baseline")
print("9. Backward propagation - 0.001737 points improvement")
print("10. Aggressive SA on small N - no improvement")
print("11. Swap moves - no improvement")
print("12. Squeeze operation - no improvement")
print("13. Ensemble from multiple CSVs - baseline is best for all N")

print("\nApproaches NOT tried:")
print("1. External datasets (bucket-of-chump, santa-2025-csv) - may have better baselines")
print("2. Much longer optimization runs (hours/days)")
print("3. Genetic algorithm with crossover between configurations")
print("4. No-Fit Polygon (NFP) based placement")
print("5. Constraint programming / MIP formulation")
print("6. Manual fine-tuning for specific N values")
print("7. Different tree rotation patterns (not just 0/180)")
print("8. Asymmetric configurations")

In [ ]:
# The key insight from the kernels is that they use external datasets
# Let's check what external datasets are referenced

print("External datasets referenced in kernels:")
print("1. jazivxt/bucket-of-chump - used by 'Why Not' kernel")
print("2. saspav/santa-2025-csv - used by 'Santa-submission' kernel")
print("")
print("These datasets likely contain pre-optimized solutions that score better than 70.68")
print("")
print("The target score of 68.92 may have been achieved by someone who:")
print("1. Ran optimization for weeks/months")
print("2. Used better starting configurations")
print("3. Used fundamentally different approaches (MIP, constraint programming)")
print("4. Combined multiple optimization techniques")
print("")
print("Without access to these external datasets, we need to:")
print("1. Run much longer optimization (hours, not minutes)")
print("2. Try fundamentally different approaches")
print("3. Focus on specific N values with most room for improvement")